In [2]:
import torch 
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0125,  0.0502, -0.1262,  0.0309, -0.2054,  0.0136, -0.0782, -0.0996,
         -0.0192,  0.0828],
        [ 0.0297, -0.0913, -0.1566, -0.1651, -0.0736,  0.0659, -0.1211, -0.0269,
          0.0159,  0.0506]], grad_fn=<AddmmBackward>)

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X)

tensor([[-0.0180,  0.2871,  0.0142,  0.0943,  0.0521,  0.0358,  0.0366,  0.0123,
          0.0190,  0.0385],
        [ 0.0232,  0.1348,  0.0904,  0.0110,  0.0773,  0.0244,  0.0733, -0.0756,
         -0.0383,  0.0244]], grad_fn=<AddmmBackward>)

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X
    
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0142,  0.0973, -0.0511, -0.0005, -0.0425,  0.4067,  0.0432,  0.0293,
          0.1067,  0.2093],
        [ 0.0143,  0.0295, -0.0603,  0.0075, -0.0540,  0.3008,  0.0620, -0.0177,
          0.1849,  0.1700]], grad_fn=<AddmmBackward>)

In [9]:
# 在正向传播函数中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        # 手写一个矩阵乘法（mm）
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
    
net = FixedHiddenMLP()
net(X)



tensor(-0.2274, grad_fn=<SumBackward0>)

In [10]:
# 混合搭配各种组合块的方法
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
                        nn.Linear(20, 64), nn.ReLU(),
                        nn.Linear(64, 32), nn.ReLU()
                    )
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1088, grad_fn=<SumBackward0>)